###  Case Study: Car Sales Data
The "Car Sales Data" dataset is a tabular dataset that contains 1.000,000 rows of information on car sales from a car dealership over the course Of a year. The dataset includes nine columns of data for each car sale. 

The first column, "Date," represents the date on which the car sale was made. The second column, "Salesperson," provides the name Of the salesperson who made the sale. The third column, "Customer Name," provides the name Of the customer who purchased the car.

The next three columns relate to the car that was sold. "Car Make" indicates the make Of the car that was purchased, such as Toyota, Honda, Ford, Chevrolet, or Nissan. "Car Model" indicates the model of the car that was purchased, such as Corolla, Civic, F-150, Silverado, or Altima. "Car Year" indicates the year Of the car that was purchased, ranging from 2010 to 2022.

The "Sale Price" column represents the sale price of the car in USD. The next column, "Commission Rate," indicates the commission rate paid to the salesperson on the sale, ranging from 0.05 to 0.15. Finally. the "Commission Earned" column provides the amount Of commission earned by the salesperson on the sale.

This dataset is useful for analyzing car sales trends over time and evaluating the performance of individual salespeople in the dealership. It can also be used for predicting sales performance and analyzing the impact Of different factors on car sales, such as car make, model, year, and salesperson performance.

Description: This dataset contains information on car sales from a car dealership over the course of a year. The dataset includes the following columns:

- Date: The date Of the car sale
- Salesperson: The name Of the salesperson who made the sale
- Customer Name: The name Of the customer who purchased the car
- Car Make: The make Of the car that was purchased
- Car Model: The model Of the car that was purchased
- Car Year: The year Of the car that was purchased
- Sale Price: The sale price Of the car in USD
- Commission Rate: The commission rate paid to the salesperson on the sale
- Commission Earned: The amount Of commission earned by the salesperson on the sale


### Factors that affect commission rate

Commission rate: date, customers name, car make, car model, car year, sale price, number of car sold of the salesperson

**Có thể áp dụng thêm:**
- Sử dụng OneHot Encoder
- Tìm hiểu GridSearchCV để chọn hyperparameters (siêu tham số) tốt nhất cho một thuật toán bất kỳ.
- Xây dựng bảng các chỉ số để chọn lựa model tốt nhất.

In [1]:
import pandas as pd
df = pd.read_csv('./datasets/car_sales_data.csv')

In [2]:
# Calculate number of car sold of the sales person

In [3]:
salesperson_counts = df.groupby('Salesperson').agg({'Date': 'count'}).reset_index()
salesperson_counts.rename(columns={'Date': 'Ncar_of_salesperson'}, inplace=True)

In [6]:
df = pd.merge(df, salesperson_counts, left_on='Salesperson', right_on = 'Salesperson', how='left')

In [ ]:
# Commission rate: date, customers name, car make, car model, car year, sale price, number of car sold of the salesperson

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
onehot = OneHotEncoder()

In [9]:
columns_to_encode = ['Customer Name','Car Year', 'Date']

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

In [10]:
columns_to_onehot_encode = ['Car Make', 'Car Model']

for column in columns_to_onehot_encode:
    new_col = pd.DataFrame(onehot.fit_transform(df[[column]]).toarray(), columns = onehot.get_feature_names_out())
    df.drop(column, axis = 1, inplace = True)
    df = df.join(new_col)

In [33]:
dep_var = 'Commission Rate'

In [34]:
ind_var = list(df.columns)

In [35]:
ind_var.remove(dep_var)

In [36]:
ind_var.remove('Commission Earned')
ind_var.remove('Salesperson')

In [37]:
ind_var

['Date',
 'Customer Name',
 'Car Year',
 'Sale Price',
 'Ncar_of_salesperson',
 'Car Make_Chevrolet',
 'Car Make_Ford',
 'Car Make_Honda',
 'Car Make_Nissan',
 'Car Make_Toyota',
 'Car Model_Altima',
 'Car Model_Civic',
 'Car Model_Corolla',
 'Car Model_F-150',
 'Car Model_Silverado']

In [38]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, train_size = 0.8, random_state=10, shuffle=True)
X_train, X_test = train_data[ind_var], test_data[ind_var]
y_train, y_test = train_data[dep_var], test_data[dep_var]

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

In [58]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [59]:
from sklearn.model_selection import GridSearchCV

In [60]:
param_grid = {'alpha':[.0001, 0.001, 0.01, 0.1, 1]}

In [61]:
grid = GridSearchCV(Ridge(), param_grid, scoring='r2')
grid.fit(X_train_scaled, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}, scoring='r2')

In [62]:
print(grid.best_params_)

{'alpha': 1}


In [63]:
from sklearn.metrics import make_scorer
import numpy as np

# Define custom MAPE scoring function
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Make it a scorer using make_scorer
mape_scorer = make_scorer(mape, greater_is_better=False)

In [64]:
grid = GridSearchCV(Lasso(), param_grid, scoring=mape_scorer)
grid.fit(X_train_scaled, y_train)

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1]},
             scoring=make_scorer(mape, greater_is_better=False))

In [65]:
print(grid.best_params_)

{'alpha': 0.0001}


In [66]:
models = {
    'Linear Regression': LinearRegression(),
    'Polynominal Features': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1),
    'Lasso Regression': Lasso(alpha=0.0001),
    'neural_network': MLPRegressor()
}

In [67]:
def model_test(model, X_train, X_test, y_train, y_test):

    model.fit(X_train, y_train)
    y_test_predicted = model.predict(X_test)
    y_train_predicted = model.predict(X_train)

    score = []
    score.append(mean_absolute_percentage_error(y_test, y_test_predicted))
    score.append(mean_absolute_percentage_error(y_train, y_train_predicted))
    score.append(mean_squared_error(y_test, y_test_predicted)**0.5)
    score.append(mean_squared_error(y_train, y_train_predicted)**0.5)
    score.append(r2_score(y_test, y_test_predicted))
    score.append(r2_score(y_train, y_train_predicted))

    return score

In [68]:
df1 = pd.DataFrame(index = ['mape_test', 'mape_train', 'rmse_test', 'rmse_train', 'R_squared_test', 'R_squared_train'])

In [69]:
for name, model in models.items():
    if name == 'Polynominal Features':
        poly_features = PolynomialFeatures(degree=2)
        X_train_poly = poly_features.fit_transform(X_train_scaled)
        X_test_poly = poly_features.fit_transform(X_test_scaled)
        df1[name] = model_test(model, X_train_poly, X_test_poly, y_train, y_test)
    else:
        df1[name] = model_test(model, X_train_scaled, X_test_scaled, y_train, y_test)

In [70]:
df1

,Linear Regression,Polynominal Features,Ridge Regression,Lasso Regression,neural_network
mape_test,0.288140,0.288134,0.288140,0.288141,0.284696
mape_train,0.287582,0.287556,0.287582,0.287583,0.284143
rmse_test,0.028871,0.028871,0.028871,0.028871,0.028896
rmse_train,0.028859,0.028858,0.028859,0.028859,0.028887
R_squared_test,0.000003,-0.000015,0.000005,-0.000001,-0.001791
R_squared_train,0.000005,0.000083,0.000005,0.000000,-0.001881
